# ```TORCH.AUTOGRAD```를 통한 자동 미분
신경망을 학습할 때 가장 자주 사용되는 알고리즘은 **역전파**이다. 매개변수(모델 가중치)는 주어진 매개변수에 대한 손실 함수의 **변화도(gradient)**에 따라 조정된다.

이러한 변화도를 계산하기 위해 Pytorch에는 ```torch.autograd```라는 자동 미분 엔진이 내장되어 있다.

In [1]:
# 입력 x, 매개변수 w와 b, 일부 손실 함수가 있는 간단한 단일 계층 신경망 가정
import torch

x = torch.ones(5) # input tensor
y = torch.zeros(3) # expected output
w = torch.randn(5, 3, requires_grad=True)
b = torch.randn(3, requires_grad=True)
z = torch.matmul(x, w)+b
loss = torch.nn.functional.binary_cross_entropy_with_logits(z, y)

## Tensor, Function과 연산그래프(Computational graph)
매개변수에 대한 손실 함수의 변화도를 계산하기 위해서는 해당 텐서에 ```requires_grad``` 속성 설정해야 한다.

In [2]:
print('Gradient function for z =', z.grad_fn)
print('Gradient function for loss =', loss.grad_fn)

Gradient function for z = <AddBackward0 object at 0x00000179B6F36610>
Gradient function for loss = <BinaryCrossEntropyWithLogitsBackward object at 0x00000179B6F36040>


## 변화도(Gradient) 계산하기
신경망에서 매개변수의 가중치를 최적화하려면 매개변수에 대한 손실함수의 도함수(derivative)를 계산해야 한다. 즉, x와 y의 일부 고정값에서 ∂loss/∂w와 ∂loss/∂b가 필요 -> ```loss.backward()```를 호출한 뒤 ```w.grad```와 ```b.grad```에서 값을 가져온다.

In [3]:
loss.backward()
print(w.grad)
print(b.grad)
# requires_grad=True라고 설정된 노드들의 grad 속성망 구할 수 있다.

tensor([[0.1620, 0.3115, 0.3264],
        [0.1620, 0.3115, 0.3264],
        [0.1620, 0.3115, 0.3264],
        [0.1620, 0.3115, 0.3264],
        [0.1620, 0.3115, 0.3264]])
tensor([0.1620, 0.3115, 0.3264])


## 변화도 추적 멈추기
```requires_grad=True```인 모든 텐서들은 연산 기록을 추적하고 변화도 계산을 지원한다. 그러나 모델을 학습한 뒤 입력 데이터를 단순히 적용하기만 하는 경우와 같이 *순전파* 연산만 필요한 경우, 이러한 추적이나 지원이 필요없을 수도 있다. 연산코드를 ```torch.no_grad()``` 블록으로 둘러싸서 연산 추적을 멈출 수 있다.

**변화도 추적을 멈춰야 하는 이유**
* 사전 학습된 신경망을 미세조정할 때 일반적으로 신경망의 일부 매개변수를 고정된 매개변수(frozen parameter)로 표시한다.
* 변화도를 추적하지 않는 텐서의 연산이 더 효율적이기 때문에, 순전파 단계만 수행할 때 연산 속도가 향상된다.

In [5]:
z = torch.matmul(x,w)+b
print(z.requires_grad)

with torch.no_grad():
    z = torch.matmul(x,w)+b
print(z.requires_grad)

True
False


In [7]:
# 텐서에 'detach()' 메소드 사용
z = torch.matmul(x,w)+b
z_det = z.detach()
print(z_det.requires_grad)

False


## 연산 그래프에 대한 추가 정보
*  autograd는 데이터(텐서)의 및 실행된 모든 연산들(및 연산 결과가 새로운 텐서인 경우도 포함하여)의 기록을 Function 객체로 구성된 방향성 비순환 그래프(DAG; Directed Acyclic Graph)에 저장(keep)
* 이 방향성 비순환 그래프(DAG)의 잎(leave)은 입력 텐서이고, 뿌리(root)는 결과 텐서이다.
* 이 그래프를 뿌리에서부터 잎까지 추적하면 연쇄 법칙(chain rule)에 따라 변화도를 자동으로 계산

*순전파 단계에서, autograd는...*

* 요청된 연산을 수행하여 결과 텐서를 계산
* DAG에 연산의 변화도 기능(gradient function)를 유지(maintain)

*역전파 단계에서는,*
DAG 뿌리(root)에서 ```.backward()```가 호출될 때 시작
* 각, ```.grad_fn```으로부터 변화도 계산
* 각 텐서의 ```.grad``` 속성에 계산 결과를 쌓고(accumulate),
* 연쇄 법칙을 사용하여, 모든 잎(leaf) 텐서들까지 전파(propagate)

-----

PyTorch에서 DAG들은 동적(dynamic)이다. 중요한 점은 그래프가 처음부터(from scratch) 다시 생성된다는 것! 매번 ```.backward()```가 호출되고 나면 autogard는 새로운 그래프를 채우기(populate) 시작한다. 이러한 점 덕분에 모델에서 흐름 제어(control flow) 구문들을 사용할 수 있게 되는 것이다. 매번 반복(iteration)할 때마다 필요하면 모양(shape)이나 크기(size), 연산(operation)을 바꿀 수 있다.


## 선택적으로 읽기(Optional Reading): 텐서 변화도와 야코비안 곱 (Jacobian Product)
대부분 스칼라 손실 함수를 가지고 일부 매개변수와 관련한 변화도를 계산해야 한다. 그러나 *출력 함수가 임의의 텐서인 경우*도 있다. 이럴 때 실제 변화도가 아닌 **야코비안 곱(Jacobian product)**을 계산한다.

야코비안 행렬 자체를 계산하는 대신, PyTorch는 주어진 입력 벡터 v=(v1…vm)에 대한 야코비안 곱(Jacobian Product) vT⋅J을 계산한다. 이 과정은 v를 인자로 backward를 호출하면 이뤄진다. v의 크기는 곱(product)을 계산하려고 하는 원래 텐서의 크기와 같아야 한다.

In [8]:
inp = torch.eye(5, requires_grad=True)
out = (inp+1).pow(2)
out.backward(torch.ones_like(inp), retain_graph=True)
print("First call\n", inp.grad)
out.backward(torch.ones_like(inp), retain_graph=True)
print("\nSecond call\n", inp.grad) # 두차례 호출하면 변화도 값이 달라진다. 이는 역방향 전파 수행 시 변화도를 누적하기 때문
inp.grad.zero_() # 제대로된 변화도 계산 위해 0으로 초기화, 실제 학습에서는 옵티마이저가 하는 일
out.backward(torch.ones_like(inp), retain_graph=True)
print("\nCall after zeroing gradients\n", inp.grad)

First call
 tensor([[4., 2., 2., 2., 2.],
        [2., 4., 2., 2., 2.],
        [2., 2., 4., 2., 2.],
        [2., 2., 2., 4., 2.],
        [2., 2., 2., 2., 4.]])

Second call
 tensor([[8., 4., 4., 4., 4.],
        [4., 8., 4., 4., 4.],
        [4., 4., 8., 4., 4.],
        [4., 4., 4., 8., 4.],
        [4., 4., 4., 4., 8.]])

Call after zeroing gradients
 tensor([[4., 2., 2., 2., 2.],
        [2., 4., 2., 2., 2.],
        [2., 2., 4., 2., 2.],
        [2., 2., 2., 4., 2.],
        [2., 2., 2., 2., 4.]])
